# Accessing PAVICS THREDDS Server


The THREDDS data storing netCDF file on PAVICS has some public and private directories. Data from public directories can be accessed anonymously, while data from private directories require authentication. This notebook shows how to access public and private data on the THREDDS server.

The PAVICS THREDDS server has a `testdata/` folder, in which we store test datasets to validate process requests. Within that directory is a `secure/` folder whose file access requires authentication (to be done).

In [1]:
# NBVAL_IGNORE_OUTPUT

# define some useful variables for following steps
import os
PAVICS_HOST = os.getenv("PAVICS_HOST", "pavics.ouranos.ca")
THREDDS_URL = "https://{}/twitcher/ows/proxy/thredds".format(PAVICS_HOST)

assert PAVICS_HOST != "", "Invalid PAVICS HOST value."
print("THREDDS URL:", THREDDS_URL)

THREDDS URL: https://host-140-10.rdext.crim.ca/twitcher/ows/proxy/thredds


First let's just open an unsecured link.

In [2]:
import xarray as xr
xr.set_options(display_style="text")  # comment out for html style, text style simpler for automated testing

PUBLIC_URL = f"{THREDDS_URL}/dodsC/birdhouse/testdata/ta_Amon_MRI-CGCM3_decadal1980_r1i1p1_199101-200012.nc"
ds = xr.open_dataset(PUBLIC_URL)
ds

<xarray.Dataset>
Dimensions:    (bnds: 2, lat: 160, lon: 320, plev: 23, time: 120)
Coordinates:
  * time       (time) datetime64[ns] 1991-01-16T12:00:00 ... 2000-12-16T12:00:00
  * plev       (plev) float64 1e+05 9.25e+04 8.5e+04 7e+04 ... 200.0 100.0 40.0
  * lat        (lat) float64 -89.14 -88.03 -86.91 -85.79 ... 86.91 88.03 89.14
  * lon        (lon) float64 0.0 1.125 2.25 3.375 ... 355.5 356.6 357.8 358.9
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) datetime64[ns] ...
    lat_bnds   (lat, bnds) float64 ...
    lon_bnds   (lon, bnds) float64 ...
    ta         (time, plev, lat, lon) float32 ...
Attributes: (12/28)
    institution:                     MRI (Meteorological Research Institute, ...
    institute_id:                    MRI
    experiment_id:                   decadal1980
    source:                          MRI-CGCM3 2011 atmosphere: GSMUV (gsmuv-...
    model_id:                        MRI-CGCM3
    forcing:                         GHG, SA, Oz, LU, Sl, Vl, BC, OC (GHG inc...
    ...                              ...
    title:                           MRI-CGCM3 model output prepared for CMIP...
    parent_experiment:               N/A
    modeling_realm:                  atmos
    realization:                     1
    cmor_version:                    2.7.1
    DODS_EXTRA.Unlimited_Dimension:  time

Now let's do the same with a secured link.

In [7]:
from webob.exc import HTTPError

SECURED_URL = f"{THREDDS_URL}/dodsC/birdhouse/testdata/secure/tasmax_Amon_MPI-ESM-MR_rcp45_r2i1p1_200601-200612.nc"
try:
    ds = xr.open_dataset(SECURED_URL, decode_cf=False)
except HTTPError as exc:  # FIXME: should be HTTPUnauthorized, but failing operation raises 500 with 'message' Unauthorized
    http_error = exc
else:
    raise RuntimeError("Expected unauthorized response, but dataset open operation did not raise!")
assert "401 Unauthorized" in str(http_error)
print("Unauthorized was raised as expected.")

Unauthorized was raised as expected.


To open a secured link, we need to open a session with `Authentication`.
Using wrong `Authentication` credentials will not work. They will raise immediately when failing login procedure.
Using valid credentials will instead succeed login, but will raise a forbidden response when attempting to retrieve
the data. Either way, user must be logged in and have appropriate access to fulfill `Authorization` requirements
of the resource.

Let's see the result when credentials are invalid.

In [9]:
import requests
from requests_magpie import MagpieAuth, MagpieAuthenticationError

BAD_USR = "an-invalid-user"
BAD_PWD = "or-bad-password"

try:
    with requests.session() as session:
        session.auth = MagpieAuth(f"https://{PAVICS_HOST}/magpie", BAD_USR, BAD_PWD)
        xr.open_dataset(SECURED_URL, decode_cf=False)  # Attributes are problematic with this file.
except (HTTPError, MagpieAuthenticationError) as exc:  # error depends on what raises (unauthorized, forbidden, login failure)
    print("Access with invalid credentials was not permitted as expected.")
else:
    raise RuntimeError("Expected authentication failure response, but login operation did not raise!")

Access with invalid credentials was not permitted as expected.


As we can see, the server identified that credentials were provided, but they were incorrect and could not log in.
Similar result would happen if login succeeded, but user was forbidden access due to insufficient permissions.

We've created an `authtest` user in advance that has access to the `secure` contents to facilitate testing.

Let's use it now to obtain the secured resource.


In [11]:
AUTH_USR = os.getenv("TEST_MAGPIE_ADMIN_USERNAME", "authtest")
AUTH_PWD = os.getenv("TEST_MAGPIE_ADMIN_PASSWORD", "authtest1234")

# Open session
with requests.Session() as session:
    session.auth = MagpieAuth(f"https://{PAVICS_HOST}/magpie", AUTH_USR, AUTH_PWD)
    # Open a PyDAP data store and pass it to xarray
    store = xr.backends.PydapDataStore.open(SECURED_URL, session=session)
    ds = xr.open_dataset(store, decode_cf=False)  # Attributes are problematic with this file.
ds

<xarray.Dataset>
Dimensions:             (bnds: 2, lat: 96, lon: 192, time: 12)
Coordinates:
  * time                (time) float64 5.699e+04 5.702e+04 ... 5.733e+04
  * lat                 (lat) float64 -88.57 -86.72 -84.86 ... 84.86 86.72 88.57
  * lon                 (lon) float64 0.0 1.875 3.75 5.625 ... 354.4 356.2 358.1
Dimensions without coordinates: bnds
Data variables:
    time_bnds           (time, bnds) float64 ...
    lat_bnds            (lat, bnds) float64 ...
    lon_bnds            (lon, bnds) float64 ...
    latitude_longitude  |S128 ...
    tasmax              (time, lat, lon) float32 ...
Attributes: (12/28)
    institution:                     Max Planck Institute for Meteorology
    institute_id:                    MPI-M
    experiment_id:                   rcp45
    source:                          MPI-ESM-MR 2011; URL: http://svn.zmaw.de...
    model_id:                        MPI-ESM-MR
    forcing:                         GHG,Oz,SD,Sl,Vl,LU
    ...                              ...
    title:                           MPI-ESM-MR model output prepared for CMI...
    parent_experiment:               historical
    modeling_realm:                  atmos
    realization:                     1
    cmor_version:                    2.6.0
    DODS_EXTRA.Unlimited_Dimension:  time

Successful listing of the above data means the user was granted access for this reference.
